In [67]:
import pandas as pd
import math
import numpy as np
import math
import random
from sklearn import datasets, linear_model, metrics
from sklearn.cross_validation import train_test_split # to split the dataset for training and testing
from sklearn.preprocessing import normalize # to normalize a matrix
from sklearn.decomposition import PCA
from sklearn.linear_model import SGDRegressor
from scipy import linalg

In [30]:
# getting the ratings data in pandas dataframe
ratings = pd.read_csv('../Yahoo_movies_multi-criteria/data_movies.txt', sep='\t',names=['user_id', 'criterion1', 'criterion2', 'criterion3', 'criterion4', 'overall', 'movie_id', 'num'])
print(max(ratings.overall)) # implies that ratings are on a scale of 13
# train, test = sample_split(ratings)
# print(pd.Series.sort_values(ratings.movie_id).unique())

13


In [31]:
# function to crop the dataset such that every item is rated by atleast ri users
def min_item_ratings_criteria(ratings, ri):
    # remove all items with number of ratings <= ri
    temp = ratings.groupby(['movie_id']).overall.count().reset_index()
#     print(temp)
    items_to_be_removed = temp[temp.overall < ri].movie_id.unique()
    print(len(items_to_be_removed))
    new_ratings = ratings[~ratings['movie_id'].isin(items_to_be_removed)]
    print(len(new_ratings.movie_id.unique()))
    return new_ratings
# min_item_ratings_criteria(5)

In [75]:
# function to crop the dataset such that ratings given by any user>=ru
def min_user_ratings_criteria(ratings, ru):
    # remove all users who have rated <= ru movies
    temp = ratings.groupby(['user_id']).overall.count().reset_index()
#     print(temp)
    users_to_be_removed = temp[temp.overall < ru].user_id.unique()
    print(len(users_to_be_removed))
    new_ratings = ratings[~ratings['user_id'].isin(users_to_be_removed)]
    print(len(new_ratings.user_id.unique()))
    return new_ratings
#     print(len(users_to_be_removed))
# min_ratings_criteria(10,1)

In [33]:
# split the datasets into smaller datasets, such that every dataset contains 'size' number of users
def split_dataset(ratings, size):
        df_group = ratings.groupby(['user_id'])
#         print(type(df_group))
        
        df_group_list = list()
        # converting the group of dataframes into list of dataframe for random shuffling
        for index, item in df_group:
            df_group_list.append(item)
        # randomly shuffling the list
        random.shuffle(df_group_list)
        no_of_datasets = int(len(df_group)/size);
#         print(no_of_datasets)
        list_datasets = list() # list containing all the small datasets
        temp = pd.DataFrame()
        count=0
        flag=0
        datasets_generated = 0 # datasets that have been generated till now
        for item in df_group_list:
            count+=1
            temp = temp.append(item)
            if count==size and datasets_generated < no_of_datasets-1:
                list_datasets.append(temp)
                print("size = ",temp.shape[0])
#                 print(temp.user_id.unique())
                temp = pd.DataFrame()
                datasets_generated += 1
                count=0
        list_datasets.append(temp) # appending the last dataset
        print("size = ",temp.shape[0])
        return list_datasets
    
# testing above function
# temp = split_dataset(ratings, 500)
# for item in temp:
#     print(len(item.user_id.unique()))

In [34]:
# Spliting a sample into groups userwise
def sample_split(dataFrame):
    df_group = dataFrame.groupby('user_id')
    train = pd.DataFrame()
    test = pd.DataFrame()
    
    for key, item in df_group:
        train, test = split_train_test(item, train, test)
    
    return train, test

In [102]:
# remove movies with <5 ratings
rating_dash = min_item_ratings_criteria(ratings, 5)
rating_dash = min_user_ratings_criteria(ratings, 5)
print("users_left = ",len(rating_dash.user_id.unique()))
# remove users who have rated <10 items
# rating_dash = min_user_ratings_criteria(rating_dash, 10)
# print(len(rating_dash.movie_id.unique()))
# list conatining all the datasets
list_datasets = split_dataset(rating_dash,1000)
# print(list_datasets[1])
# print(rating_dash.shape)

69
907
0
6078
users_left =  6078
size =  10582
size =  10203
size =  9915
size =  10397
size =  10379
size =  10680


In [36]:
# spliting sample in 70% training data and 30% testing data
def split_train_test(dataFrame, train, test):
    temp_train, temp_test = train_test_split(dataFrame, test_size = 0.3, random_state=1212)# in this our main data is split into train and test
    # the attribute test_size=0.3 splits the data into 70% and 30% ratio. train=70% and test=30%
    train = train.append(temp_train)
    test = test.append(temp_test)
    
    return train, test

In [37]:
# splitting into train and test data
train = list()
test = list()
num_of_datasets = len(list_datasets)
print(num_of_datasets)
for i in range(num_of_datasets):
    print(i)
    t1, t2 = sample_split(list_datasets[i])
    train.append(t1)
    test.append(t2)

6
0
1
2
3
4
5


In [103]:
# applying linear regression on list_datasets[0]
# print(list_datasets[0])
for i in range(len(list_datasets)):
    df = list_datasets[i]
    df_group = df.groupby(['user_id'])
    count = 0
    for key,item in df_group:
    #     print(item.shape[0])
        # splitting the item in train and test data
        train, test = train_test_split(item, test_size = 0.2, random_state=1212)
        train_X = train[['criterion1', 'criterion2', 'criterion3', 'criterion4']]
        train_y = train.overall
        test_X = test[['criterion1', 'criterion2', 'criterion3', 'criterion4']]
        test_y = test.overall
        # create linear regression object
        clf = linear_model.LinearRegression(normalize=True)
        # train the model using the training sets
        clf.fit(train_X, train_y)
        prediction = clf.predict(test_X)
        score = clf.score(test_X, test_y)
        if(score<=0):
    #         print(train,"\n", test)
    #         print(prediction)
            count+=1
    #         print(item.shape[0])
    #     print("accuracy: ", score)
    print(count)
#     print(train_X)

532
555
538
582
589
638
